# DeepLabCut 3 (SuperModel) — Output Analyzer

This notebook discovers and analyzes the output files produced by DeepLabCut 3 (SuperModel) when you analyze a video.

**What you'll get:**
- Automatic file discovery (HDF5 predictions, labeled videos, before/after adapt JSONs, model snapshots)
- Robust HDF5 loader that handles multiple DLC schema variants
- Video-specific tracking quality metrics derived from likelihoods
- Diagnostics plots (likelihoods, per-bodypart coverage, inter-frame motion, smoothness)
- JSON diff of `*_before_adapt.json` vs `*_after_adapt.json`
- Final summary report cell you can export/print

> Tip: Run cells from top to bottom. Set `BASE_DIR` below to where your DLC outputs live.


In [ ]:
# ✅ Configure this path to the folder that contains your DLC 3 SuperModel outputs
from pathlib import Path
BASE_DIR = Path('.')  # <- change me to your output folder if needed, e.g., Path('/Users/you/experiments/run_01')

# Optional: If your outputs are scattered, you can add extra directories here:
EXTRA_DIRS = []  # e.g., [Path('/another/folder/of/outputs')]

# Likelihood threshold(s) used for quality metrics
LIKELIHOOD_THRESHOLDS = [0.9, 0.8, 0.5]

# If you have an expected frame rate (fps) for speed estimates, set it here; otherwise we infer if possible
FPS_HINT = None


In [ ]:
import os, re, sys, json, math, textwrap, itertools
from pathlib import Path
import numpy as np
import pandas as pd
import h5py

# Plotting (matplotlib only, one chart per figure, default colors)
import matplotlib.pyplot as plt

try:
    import cv2
except Exception as e:
    cv2 = None

def find_output_files(base: Path, extras=()):
    bases = [base] + list(extras)
    files = []
    for b in bases:
        if b.exists():
            for p in b.rglob('*'):
                if p.is_file():
                    files.append(p)
    # Classify
    h5_preds = [p for p in files if p.suffix.lower()=='.h5' and 'snapshot' not in p.name.lower()]
    h5_snapshots = [p for p in files if p.suffix.lower()=='.h5' and 'snapshot' in p.name.lower()]
    jsons = [p for p in files if p.suffix.lower()=='.json']
    labeled_vids = [p for p in files if p.suffix.lower() in ('.mp4','.avi','.mov') and 'labeled' in p.stem.lower()]
    other_vids = [p for p in files if p.suffix.lower() in ('.mp4','.avi','.mov') and 'labeled' not in p.stem.lower()]
    eval_candidates = [p for p in files if p.name.lower().startswith('evaluation-results') and p.suffix.lower() in ('.pkl','.csv','.json')]
    return {
        'all': files,
        'h5_predictions': h5_preds,
        'h5_snapshots': h5_snapshots,
        'jsons': jsons,
        'labeled_videos': labeled_vids,
        'source_videos': other_vids,
        'eval_results': eval_candidates
    }

def read_any_h5_predictions(path: Path):
    """Robustly read DLC predictions from HDF5, returning (df, meta) where df has columns:
        MultiIndex or flat: (bodypart, coord) or bodypart_coord
        coords are typically ['x','y','likelihood'] (p)
    """
    meta = {}
    # Try pandas first (common for DLC)
    try:
        with pd.HDFStore(path, 'r') as store:
            keys = store.keys()
            meta['keys'] = list(keys)
            # Heuristic: pick the largest key table
            sizes = {}
            for k in keys:
                try:
                    n = store.get_storer(k).nrows
                except Exception:
                    n = None
                sizes[k] = n
            # prefer '/df_with_missing' then largest
            key = '/df_with_missing' if '/df_with_missing' in keys else max(keys, key=lambda k: sizes.get(k, -1))
            df = store.select(key)
            meta['selected_key'] = key
    except Exception as e:
        # Fallback: use h5py + manual conversion
        with h5py.File(path, 'r') as f:
            meta['h5py_keys'] = list(f.keys())
            # Try common dataset name guesses
            candidate = None
            for guess in ['df_with_missing','table','data','/','/df_with_missing']:
                if guess in f:
                    candidate = guess
                    break
            if candidate is None:
                # pick first dataset-ish
                for k in f.keys():
                    if isinstance(f[k], h5py.Dataset):
                        candidate = k
                        break
            if candidate is None:
                raise RuntimeError(f"Could not find a dataset inside {path}")
            data = f[candidate][()]
            df = pd.DataFrame(data)
            meta['selected_key'] = candidate
    # Normalize columns
    if isinstance(df.columns, pd.MultiIndex):
        cols = df.columns
    else:
        # Attempt to parse 'bodypart_x' style
        new_cols = []
        for c in df.columns:
            s = str(c)
            if any(s.endswith('_'+coord) for coord in ['x','y','likelihood','p','prob','conf','confidence']):
                for coord in ['x','y','likelihood','p','prob','conf','confidence']:
                    if s.endswith('_'+coord):
                        bp = s[:-(len(coord)+1)]
                        new_cols.append((bp, 'likelihood' if coord in ['p','prob','conf','confidence'] else coord))
                        break
            else:
                # keep as is under 'meta'
                new_cols.append(('meta', s))
        df.columns = pd.MultiIndex.from_tuples(new_cols)
    # Unify likelihood label
    def _norm_coord(c):
        return 'likelihood' if c in ['p','prob','conf','confidence'] else c
    df.columns = pd.MultiIndex.from_tuples([(bp, _norm_coord(coord)) for bp,coord in df.columns])
    return df, meta

def estimate_fps_from_video(video_path: Path):
    if cv2 is None:
        return None
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        return None
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    if fps and fps > 0:
        return fps
    return None

def pairwise_diff(a):
    return np.abs(np.diff(a.astype(float), axis=0))

def smoothness_metric(series, window=5):
    # Simple smoothness: mean absolute second derivative magnitude
    s = np.array(series, dtype=float)
    if len(s) < window+2:
        return np.nan
    v = np.diff(s)
    a = np.diff(v)
    return float(np.nanmean(np.abs(a)))


In [ ]:
files = find_output_files(BASE_DIR, EXTRA_DIRS)
print('Found:')
for k,v in files.items():
    print(f'  {k}: {len(v)}')
    for p in v[:5]:
        print('   -', p)
    if len(v)>5:
        print('   ...')


## Inspect model metadata (before/after adapt JSON)

In [ ]:
import json, difflib

before_json = [p for p in files['jsons'] if 'before_adapt' in p.name]
after_json  = [p for p in files['jsons'] if 'after_adapt'  in p.name]

def load_json_safe(p):
    try:
        return json.loads(Path(p).read_text())
    except Exception as e:
        return {'_error': str(e)}

before = load_json_safe(before_json[0]) if before_json else None
after  = load_json_safe(after_json[0]) if after_json else None

print('Before adapt JSON:', before_json[0] if before else 'None')
print('After adapt  JSON:', after_json[0] if after  else 'None')

if before and after:
    before_txt = json.dumps(before, indent=2, sort_keys=True)
    after_txt  = json.dumps(after,  indent=2, sort_keys=True)
    diff = list(difflib.unified_diff(before_txt.splitlines(), after_txt.splitlines(), lineterm=''))
    print('\nUnified diff (truncated to 300 lines):')
    for line in diff[:300]:
        print(line)


## Load prediction HDF5 files

In [ ]:
pred_handles = []
for h5 in files['h5_predictions']:
    try:
        df, meta = read_any_h5_predictions(h5)
        # Heuristic: drop all-'meta' columns if any
        if all(bp=='meta' for bp,_ in df.columns):
            continue
        pred_handles.append((h5, df, meta))
        print(f'Loaded: {h5}  | shape={df.shape}  | key={meta.get("selected_key")}')
    except Exception as e:
        print(f'Failed to load {h5}: {e}')

if not pred_handles:
    print('No prediction HDF5 files could be loaded. Check BASE_DIR or file formats.')


## Compute video-specific quality metrics

In [ ]:
summary_rows = []
per_bp_rows = []

for h5, df, meta in pred_handles:
    # Identify coords
    coords = sorted(set(c for _,c in df.columns))
    # Expect x,y,likelihood
    if 'likelihood' not in coords:
        print(f'Warning: no likelihood column detected in {h5.name}; metrics will be limited.')
    n_frames = len(df)
    bodyparts = sorted(set(bp for bp,_ in df.columns if bp!='meta'))

    # Basic stats
    for bp in bodyparts:
        cols = df[bp]
        has_like = 'likelihood' in cols
        mean_like = float(cols['likelihood'].mean()) if has_like else np.nan
        med_like  = float(cols['likelihood'].median()) if has_like else np.nan
        coverage = {}
        if has_like:
            for thr in LIKELIHOOD_THRESHOLDS:
                coverage[thr] = float((cols['likelihood']>=thr).mean())
        # Inter-frame motion (pixels/frame)
        if {'x','y'}.issubset(cols.columns):
            xy = cols[['x','y']].values
            diffs = pairwise_diff(xy)
            speed_pf = np.linalg.norm(diffs, axis=1)  # pixels per frame
            mean_speed = float(np.nanmean(speed_pf))
            smoothness = smoothness_metric(cols['x']) + smoothness_metric(cols['y'])
        else:
            mean_speed = np.nan
            smoothness = np.nan

        per_bp_rows.append({
            'file': h5.name,
            'frames': n_frames,
            'bodypart': bp,
            'mean_likelihood': mean_like,
            'median_likelihood': med_like,
            **{f'coverage_ge_{thr}': coverage.get(thr, np.nan) for thr in LIKELIHOOD_THRESHOLDS},
            'mean_speed_px_per_frame': mean_speed,
            'smoothness_metric': smoothness
        })

    # File-level summary
    mean_like_all = float(np.nanmean([r['mean_likelihood'] for r in per_bp_rows if r['file']==h5.name]))
    summary_rows.append({
        'file': h5.name,
        'frames': n_frames,
        'n_bodyparts': len(bodyparts),
        'mean_likelihood_all_bps': mean_like_all
    })

summary_df = pd.DataFrame(summary_rows)
per_bp_df = pd.DataFrame(per_bp_rows)

from caas_jupyter_tools import display_dataframe_to_user
if not summary_df.empty:
    display_dataframe_to_user('DLC3 summary by file', summary_df)
if not per_bp_df.empty:
    display_dataframe_to_user('DLC3 per-bodypart metrics', per_bp_df)


## Plots

In [ ]:
# Likelihood distribution per file (all bodyparts pooled)
for h5, df, meta in pred_handles:
    like_cols = [df[bp]['likelihood'] for bp,_c in df.columns.unique(level=0) if bp!='meta' and 'likelihood' in df[bp]]
    if not like_cols:
        continue
    like = pd.concat(like_cols, axis=0).dropna()
    plt.figure()
    like.hist(bins=50)
    plt.title(f'Likelihood distribution — {h5.name}')
    plt.xlabel('likelihood')
    plt.ylabel('count')
    plt.show()


In [ ]:
# Coverage per bodypart at threshold 0.9 (first file only for simplicity)
if pred_handles:
    h5, df, meta = pred_handles[0]
    bp_cov = []
    for bp in sorted(set(bp for bp,_ in df.columns if bp!='meta')):
        if 'likelihood' in df[bp]:
            cov = float((df[bp]['likelihood']>=0.9).mean())
            bp_cov.append((bp, cov))
    if bp_cov:
        labels, vals = zip(*bp_cov)
        plt.figure()
        plt.bar(range(len(vals)), vals)
        plt.xticks(range(len(vals)), labels, rotation=45, ha='right')
        plt.title(f'Coverage (likelihood ≥ 0.9) per bodypart — {h5.name}')
        plt.ylabel('fraction of frames')
        plt.tight_layout()
        plt.show()


In [ ]:
# Inter-frame displacement (speed) over time for a chosen bodypart (first file, first bp)
if pred_handles:
    h5, df, meta = pred_handles[0]
    bodyparts = sorted(set(bp for bp,_ in df.columns if bp!='meta'))
    if bodyparts and {'x','y'}.issubset(df[bodyparts[0]].columns):
        xy = df[bodyparts[0]][['x','y']].values
        spf = np.linalg.norm(np.diff(xy, axis=0), axis=1)
        plt.figure()
        plt.plot(spf)
        plt.title(f'Inter-frame displacement (pixels/frame) — {h5.name} — {bodyparts[0]}')
        plt.xlabel('frame')
        plt.ylabel('pixels/frame')
        plt.show()


## (Optional) Infer FPS from labeled video and add px/s speed

In [ ]:
fps = None
if files['labeled_videos']:
    fps = estimate_fps_from_video(files['labeled_videos'][0])
if fps is None:
    fps = FPS_HINT
print('FPS used:', fps)

if fps and pred_handles:
    h5, df, meta = pred_handles[0]
    bodyparts = sorted(set(bp for bp,_ in df.columns if bp!='meta'))
    if bodyparts and {'x','y'}.issubset(df[bodyparts[0]].columns):
        xy = df[bodyparts[0]][['x','y']].values
        spf = np.linalg.norm(np.diff(xy, axis=0), axis=1)  # pixels/frame
        sps = spf * fps
        plt.figure()
        plt.plot(sps)
        plt.title(f'Estimated speed (pixels/second) — {h5.name} — {bodyparts[0]}')
        plt.xlabel('frame')
        plt.ylabel('pixels/second')
        plt.show()


## (Optional) Model evaluation results (if available)

In [ ]:
eval_rows = []
for p in files['eval_results']:
    try:
        if p.suffix.lower()=='.csv':
            edf = pd.read_csv(p)
            edf['_source'] = p.name
            eval_rows.append(edf)
        elif p.suffix.lower()=='.json':
            with open(p) as f:
                data = json.load(f)
            edf = pd.json_normalize(data)
            edf['_source'] = p.name
            eval_rows.append(edf)
        else:
            # pkl – try pandas
            edf = pd.read_pickle(p)
            if isinstance(edf, pd.DataFrame):
                edf['_source'] = p.name
                eval_rows.append(edf)
            else:
                edf = pd.DataFrame([{'_raw': str(edf), '_source': p.name}])
                eval_rows.append(edf)
        print('Loaded evaluation results:', p.name)
    except Exception as e:
        print('Failed to read eval results', p, e)

if eval_rows:
    eval_df = pd.concat(eval_rows, ignore_index=True, sort=False)
    from caas_jupyter_tools import display_dataframe_to_user
    display_dataframe_to_user('DLC3 evaluation results (if any)', eval_df)
else:
    print('No evaluation results files found.')


## Final summary (printable)

In [ ]:
report_lines = []
report_lines.append('# DLC3 (SuperModel) — Analysis Report')
report_lines.append('')
report_lines.append('## Files discovered')
for k in ['h5_predictions','jsons','labeled_videos','h5_snapshots','eval_results']:
    report_lines.append(f'### {k}')
    for p in files[k]:
        report_lines.append(f'- {p.name}')
    if not files[k]:
        report_lines.append('- (none)')
report_lines.append('')

if not summary_df.empty:
    report_lines.append('## File-level quality (likelihood-based)')
    for _,row in summary_df.iterrows():
        report_lines.append(f"- **{row['file']}**: frames={int(row['frames'])}, bodyparts={int(row['n_bodyparts'])}, mean likelihood (all bps)={row['mean_likelihood_all_bps']:.3f}")
    report_lines.append('')

if not per_bp_df.empty:
    report_lines.append('## Per-bodypart metrics (first few rows)')
    head = per_bp_df.head(12)
    report_lines.append(head.to_markdown(index=False))

if before and after:
    report_lines.append('')
    report_lines.append('## Notable changes after adaptation (JSON diff present above)')
else:
    report_lines.append('')
    report_lines.append('## Adaptation JSONs not both present; skip diff.')

print('\n'.join(report_lines))
